In [1]:
import os
import praw
import pandas as pd
import datetime as dt
import logging
import numpy as np
from numpy import random
import gensim
import nltk
nltk.download('all')
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import classification_report
from sklearn.linear_model import SGDClassifier
import pickle

from sklearn.cross_validation import train_test

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /home/pranay/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /home/pranay/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /home/pranay/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to
[nltk_data]    |     /home/pranay/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to
[nltk_data]    |     /home/pranay/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to
[nltk_data]    |     /home/pranay/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-date!
[nltk_data]    | Downloading package

[nltk_data]    |   Package reuters is already up-to-date!
[nltk_data]    | Downloading package rte to /home/pranay/nltk_data...
[nltk_data]    |   Package rte is already up-to-date!
[nltk_data]    | Downloading package semcor to
[nltk_data]    |     /home/pranay/nltk_data...
[nltk_data]    |   Package semcor is already up-to-date!
[nltk_data]    | Downloading package senseval to
[nltk_data]    |     /home/pranay/nltk_data...
[nltk_data]    |   Package senseval is already up-to-date!
[nltk_data]    | Downloading package sentiwordnet to
[nltk_data]    |     /home/pranay/nltk_data...
[nltk_data]    |   Package sentiwordnet is already up-to-date!
[nltk_data]    | Downloading package sentence_polarity to
[nltk_data]    |     /home/pranay/nltk_data...
[nltk_data]    |   Package sentence_polarity is already up-to-date!
[nltk_data]    | Downloading package shakespeare to
[nltk_data]    |     /home/pranay/nltk_data...
[nltk_data]    |   Package shakespeare is already up-to-date!
[nltk_data]    

ModuleNotFoundError: No module named 'sklearn.cross_validation'

In [2]:
flares = ["AskIndia", "Non-Political", "[R]eddiquette", "Scheduled", "Photography", "Science/Technology", "Politics", "Business/Finance", "Policy/Economy", "Sports", "Food", "AMA"]


rindi = pd.read_csv('Data/cleaned.csv')

In [7]:
def get_date(created):
    return dt.datetime.fromtimestamp(created)

def string_form(value):
    return str(value)

def clean_text(text):
     from sklearn.linear_model import SGDClassifier
     text = BeautifulSoup(text).text
     text = text.lower()
     text = REPLACE_BY_SPACE_RE.sub(' ', text)
     text = BAD_SYMBOLS_RE.sub('', text)
     text = ' '.join(word for word in text.split() if word not in STOPWORDS)
     return text
    

In [9]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

rindi['title'] = rindi['title'].apply(string_form)
rindi['body'] = rindi['body'].apply(string_form)
rindi['comments'] = rindi['comments'].apply(string_form)

rindi['title'] = rindi['title'].apply(clean_text)
rindi['body'] = rindi['body'].apply(clean_text)
rindi['comments'] = rindi['comments'].apply(clean_text)

feature_combine = rindi["title"] + rindi["comments"] + rindi["url"] + rindi['body']
rindi = rindi.assign(feature_combine = feature_combine)

In [11]:
X = rindi.feature_combine
y = rindi.flair


sgd = Pipeline([('vect', CountVectorizer()),
                  ('tfidf', TfidfTransformer()),
                  ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)),
                 ])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state = 42)

sgd.fit(X_train, y_train)

y_pred = sgd.predict(X_test)


#print('accuracy %s' % accuracy_score(y_pred, y_test))

In [12]:
pickle.dump(sgd, open('model1.pkl','wb'))